<a href="https://colab.research.google.com/github/AminHonarmandiShandiz/LLMS/blob/main/HelloAIProgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

Load the Pre-trained Model

In [ ]:
from transformers import AutoModel, AutoTokenizer

# Load the pre-trained model and tokenizer
model_name = 'bert-base-uncased'
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Customize Input Text Length

In [ ]:
max_length = 128  # Customize as needed

text = "Your input text goes here."
inputs = tokenizer(text, max_length=max_length, truncation=True, padding='max_length', return_tensors='pt')


Extract Text Embeddings

In [ ]:
import torch
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state  # Shape: (batch_size, seq_length, hidden_size)

# Optionally, average over the sequence length to get a single vector per input text
text_embedding = embeddings.mean(dim=1)  # Shape: (batch_size, hidden_size)


In [ ]:
print((text_embedding.shape))

torch.Size([1, 768])


Customize Embedding Dimensions


In [ ]:
import torch.nn as nn

new_embedding_dim = 128  # Customize as needed
embedding_layer = nn.Linear(model.config.hidden_size, new_embedding_dim)

text_embedding = embedding_layer(text_embedding)


Load a Decoder Model for Report Generation

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

decoder_model_name = 'gpt2'
decoder_model = AutoModelForCausalLM.from_pretrained(decoder_model_name)
decoder_tokenizer = AutoTokenizer.from_pretrained(decoder_model_name)


Prepare Decoder Input

In [ ]:
decoder_input_text = "[CLS] Start of report: "
decoder_inputs = decoder_tokenizer(decoder_input_text, return_tensors='pt')


Generate Text

In [ ]:
max_output_length = 100  # Customize as needed

# Generate text
outputs = decoder_model.generate(
    decoder_inputs['input_ids'],
    max_length=max_output_length,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

# Decode the output text
generated_text = decoder_tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Report:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Report: [CLS] Start of report: 000000000000000000000000000000000000000000000001

C:\Program Files (x86)\Steam\steamapps\common\Counter-Strike Global Offensive\sdk_content\maps\test_2.bsp (PUP.Optional.SmartBar.A) -> Quarantined and deleted successfully. C:\Users\marcus\AppData\Local\Smartbar\Common\iconsWide\d3d9c9f-e9d0-4


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, GPT2LMHeadModel, GPT2Tokenizer

def load_and_custom_model(bert_model_name, gpt_model_name, max_input_length, new_embedding_dim, max_output_length):
    # Load BERT model and tokenizer for text embeddings
    bert_model = AutoModel.from_pretrained(bert_model_name)
    bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

    # Load GPT-2 model and tokenizer for text generation
    gpt_model = GPT2LMHeadModel.from_pretrained(gpt_model_name)
    gpt_tokenizer = GPT2Tokenizer.from_pretrained(gpt_model_name)
    gpt_tokenizer.pad_token = gpt_tokenizer.eos_token  # Ensure GPT-2 has a pad token

    # Tokenize input text for BERT
    text = "What are the symptoms of COVID"
    inputs = bert_tokenizer(text, max_length=max_input_length, truncation=True, padding='max_length', return_tensors='pt')

    # Extract text embeddings from BERT
    with torch.no_grad():
        outputs = bert_model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling to get a single embedding

    # Customize embedding dimensions
    embedding_layer = torch.nn.Linear(bert_model.config.hidden_size, new_embedding_dim)
    custom_embeddings = embedding_layer(embeddings)

    # Prepare embeddings for GPT-2
    batch_size = custom_embeddings.size(0)
    expanded_embeddings = custom_embeddings.unsqueeze(1).repeat(1, max_output_length, 1)  # Repeat embeddings for sequence length
    gpt_embeddings = torch.cat([expanded_embeddings, torch.zeros(batch_size, max_output_length, gpt_model.config.n_embd - new_embedding_dim)], dim=2)

    # Generate text with GPT-2 using BERT embeddings as context
    gpt_model_inputs = gpt_embeddings.to(gpt_model.device)  # Ensure embeddings are on the same device as GPT-2
    attention_mask = torch.ones(gpt_model_inputs.size()[:-1], device=gpt_model.device)  # Attention mask for generated text

    # Generate text
    generated_ids = gpt_model.generate(
        inputs_embeds=gpt_model_inputs,
        attention_mask=attention_mask,
        max_length=max_output_length + gpt_model_inputs.size(1),  # Adjust max_length to include input embeddings length
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True,
        pad_token_id=gpt_tokenizer.pad_token_id
    )

    # Decode the output text
    generated_text = gpt_tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return custom_embeddings, generated_text

# Usage
custom_embeddings, generated_report = load_and_custom_model(
    'emilyalsentzer/Bio_ClinicalBERT', 'gpt2', 512, 512, 200
)
#print("Custom Embeddings:", custom_embeddings)
print("Generated Report:", generated_report)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Report: .

, and the same as it is in the other way it was in that it, or a copy of that is not be able to do anything else, as long as well as you do to the player's own team). If there are no more than one of your opponent's life total of any number of cards in a game, but not for the purpose of the next turn it has the opponent has in your deck). The next time you can't be used in any way to make the game a little bit easier to get the most out of it. This is a bit of a lot more fun to melding with other players. It's a good idea is to have the best of all the time it's there is. I don't want to see through the whole thing, so I'm not sure if you're not a part of what I do not work in this game and I have no idea of) and so on the way that I can see it through


In [ ]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 32.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSeq2SeqLM
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

In [ ]:
model_name_or_path = 'bigscience/mt0-large'

In [ ]:
peft_config=LoraConfig(
    task_type= TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

In [ ]:
model=AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

config.json:   0%|          | 0.00/800 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 1,231,940,608 || trainable%: 0.1915


 Customize Output Text Length - adjust as needed